# Метод внешнего штрафа

Реализация метода внешнего штрафа с применением градиентного спуска *с постоянным шагом* для поиска минимума функции вида ${f(x, y)=Ax^2+By^2}$ до заданной точности ${\varepsilon}$.

In [1]:
A = 1
B = 9
EPS = 0.01

$${f(x, y) = x^2 + 9y^2}$$

In [2]:
def func(x, y):
    return A*x**2+B*y**2

*i-ое ограничение* мнимизируемой функции представляет собой функцию вида ${f_i(x, y)\leq0}$, где ${1\leq{i}\leq{n} = 2}$.
\begin{cases}{f_1(x,y)=x+2y+1}\\
{f_2(x,y)=1-x-y}\end{cases}

In [3]:
def cond1(x, y):
    return C1_B*x+C1_B*y+C1_C

def cond2(x, y):    
    return C2_C-C2_A*x-C2_B*y

def check_condition(cond):
    return cond <= 0

$${S_\gamma = f(x, y)+\gamma\left(C_1\left[\max(0, f_1(x, y))\right]^P+\ldots+C_n\left[\max(0, f_n(x, y))\right]^P\right)}$$

In [4]:
P = 2
C1, C2 = 1, 1
C1_A, C1_B, C1_C = 1, 2, 1
C2_A, C2_B, C2_C = -1, -1, 1

$${S_\gamma = x^2+9y^2+\gamma\left(\left[\max(0, x+2y+1)\right]^2+\left[\max(0, 1-x-y)\right]^2\right)}$$

In [5]:
def s_gamma(x, y, gamma):
    return func(x, y) + gamma*(C1*max(0, condition1)**P + C2*max(0, condition2)**P)

$${\nabla{S_\gamma(x, y)}}$$

In [6]:
def gamma_gradient(x, y, gamma):
    if check_condition(cond1(x,y)) and check_condition(cond2(x,y)):
        return 2*A*x, 2*B*y
    elif not check_condition(cond1(x,y)) and check_condition(cond2(x,y)):
        return 2*A*x + 2*C1_A*C1*gamma*(C1_C + C1_A*x + C1_B*y), 2*B*y + 2*C1_B*C1*gamma*(C1_C + C1_A*x + C1_B*y)
    elif check_condition(cond1(x,y)) and not check_condition(cond2(x,y)):
        return 2*A*x + 2*gamma*C2_A*C2*(C2_C + C2_A*x + C2_B*y), 2*B*y + 2*gamma*C2_B*C2*(C2_C + C2_A*x + C2_B*y)
    else:
        return 2*(A*x + gamma*C1_A*C1*(C1_A*x + C1_B*y + C1_C) + gamma*C2_A*C2*(C2_A*x + C2_B*y + C2_C)), \
    2*(B*y + gamma*C1_B*C1*(C1_A*x + C1_B*y + C1_C) + gamma*C2_B*C2*(C2_A*x + C2_B*y + C2_C))

$${||\overline{v}|| = \sqrt{x^2 + y^2}}$$

In [7]:
def vect_norm(x, y):
    return (x**2+y**2)**0.5

$$\left(\begin{array}{ccc}x^{k+1}\\y^{k+1}\\\end{array}\right)=
\left(\begin{array}{ccc}x^k\\y^k\\\end{array}\right) - {1\over L}\nabla S_\gamma(x^k, y^k)$$
Условие остановки:$${||\nabla{S_\gamma(x^*, y^*)}||<\varepsilon}$$

In [8]:
def gradient_descent(x, y, gamma):
    i = 0
    s = step(gamma) # получаем шаг
    a, b = gamma_gradient(x, y, gamma) # получаем градиент 
    while vect_norm(a, b) > EPS:
        x, y = x - a/s, y - b/s
        a, b = gamma_gradient(x, y, gamma)
        i += 1
    return x, y, i

Шаг градиента вычисляется по формуле $${L = \max(H(x), H(y))},$$где $H(t)$ - коэффициент при $t^2$ функции ${S_\gamma(x, y)}$ в точке, нарушающей *оба* ограничения.

In [9]:
def step(gamma):
    return 2*max(A+gamma*(C1_A**2+C2_A**2), B+gamma*(C1_B**2+C2_B**2))

In [10]:
from random import uniform

def generate_point(a, b):
    for i in range(10):
        x = uniform(a, b)
        y = uniform(a, b)
        if not check_condition(cond1(x, y)) and not check_condition(cond2(x, y)):
            return x, y
    raise Exeption('Bad point generated')

### Демонстрация работы метода

In [17]:
x, y = generate_point(-5, 5)
g = 1
builder = '['
print('+-------+-------+')
print('|γ\t|i\t|')
print('+-------+-------+')

for i in range(1, 11):
    res = gradient_descent(x, y, g)
    print('|%d \t|%d\t|' % (g, res[2]))
    builder+="(%f, %f), " % (res[0], res[1])
    g*=2
    
print('+-------+-------+')
builder = builder[:-2] + ']'

+-------+-------+
|γ	|i	|
+-------+-------+
|1 	|51	|
|2 	|63	|
|4 	|91	|
|8 	|143	|
|16 	|237	|
|32 	|393	|
|64 	|654	|
|128 	|1113	|
|256 	|1973	|
|512 	|3644	|
+-------+-------+


In [18]:
print(builder)

[(0.088957, -0.090411), (0.201768, -0.168892), (0.409039, -0.307122), (0.746074, -0.528643), (1.212499, -0.833680), (1.736049, -1.175493), (2.203006, -1.480166), (2.541672, -1.701078), (2.752239, -1.838415), (2.870879, -1.915791)]
